# Upload DIVERSE Model Logs to HuggingFace

DIVERSE model training loglarını HuggingFace'e yükler

In [ ]:
# Kurulum
!pip install -q huggingface_hub

from google.colab import drive
from huggingface_hub import HfApi, create_repo
import os

drive.mount('/content/drive')
print("✓ Drive bağlandı")

In [ ]:
# HuggingFace Login
from huggingface_hub import notebook_login

notebook_login()
print("✓ HuggingFace'e giriş yapıldı")

In [ ]:
# DIVERSE Model için repo hazırla
api = HfApi()

try:
    create_repo(
        repo_id="bodhai/qwen-coder-lora-diverse",
        repo_type="model",
        exist_ok=True
    )
    print("✓ DIVERSE repo hazır")
except Exception as e:
    print(f"✓ DIVERSE repo zaten var: {e}")

In [ ]:
# DIVERSE logs kontrol ve görselleştirme
import json
import matplotlib.pyplot as plt

diverse_log_dir = '/content/drive/MyDrive/lora_checkpoints/diverse/logs'

print(f"📂 Log klasörü: {diverse_log_dir}")
print("⚠️  Bu notebook DIVERSE modelinin training yapıldığı Drive'da çalıştırılmalı!\n")

if os.path.exists(diverse_log_dir):
    # Dosyaları listele
    files = [f for f in os.listdir(diverse_log_dir) if f.endswith('.jsonl')]
    print(f"✓ Bulunan log dosyaları ({len(files)} adet):")
    for f in files:
        size = os.path.getsize(os.path.join(diverse_log_dir, f)) / 1024
        print(f"  - {f} ({size:.1f} KB)")
    
    # Log'ları oku ve görselleştir
    if files:
        log_file = os.path.join(diverse_log_dir, files[0])
        with open(log_file, 'r') as f:
            logs = [json.loads(line) for line in f]
        
        train_logs = [log for log in logs if 'train_loss' in log]
        
        if train_logs:
            steps = [log['step'] for log in train_logs]
            losses = [log['train_loss'] for log in train_logs]
            
            # Basit grafik
            plt.figure(figsize=(10, 4))
            plt.plot(steps, losses, '#10AC84', linewidth=2, alpha=0.7)
            plt.xlabel('Step', fontsize=12, fontweight='bold')
            plt.ylabel('Loss', fontsize=12, fontweight='bold')
            plt.title('DIVERSE Model - Training Loss', fontsize=14, fontweight='bold')
            plt.grid(True, alpha=0.3)
            plt.tight_layout()
            plt.show()
            
            print(f"\n📊 İstatistikler:")
            print(f"  Toplam Step: {len(train_logs)}")
            print(f"  İlk Loss: {losses[0]:.4f}")
            print(f"  Son Loss: {losses[-1]:.4f}")
            print(f"  İyileşme: {((losses[0] - losses[-1]) / losses[0] * 100):.1f}%")
else:
    print(f"❌ Log klasörü bulunamadı: {diverse_log_dir}")
    print("\n🔍 Drive yapısını kontrol et:")
    !ls -la /content/drive/MyDrive/lora_checkpoints/
    print("\n💡 Bu Drive'da DIVERSE modeli yok mu? Doğru Drive'ı bağladığından emin ol!")

In [ ]:
# HuggingFace'e yükle
if os.path.exists(diverse_log_dir):
    print("📤 HuggingFace'e yükleniyor...\n")
    
    api.upload_folder(
        folder_path=diverse_log_dir,
        repo_id="bodhai/qwen-coder-lora-diverse",
        repo_type="model",
        path_in_repo="logs"
    )
    
    print("\n✅ DIVERSE logs başarıyla yüklendi!")
    print("\n🔗 Kontrol et: https://huggingface.co/bodhai/qwen-coder-lora-diverse/tree/main/logs")
else:
    print("❌ Yükleme yapılamadı - log klasörü bulunamadı")